In [65]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from numpy.core.fromnumeric import size
from sklearn.svm import SVR
from torch.optim import Adam
from torch.autograd import Variable
from sklearn import preprocessing as pre
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_percentage_error

# SVR

In [46]:
def shiftData(from_, to_, feature):
    for i in range(from_, to_ + 1):
        df['prev_' + feature + str(i)] = df[feature].shift(periods=i)
    return df.dropna().copy()

In [47]:
dataPath = '/Users/cicada/Documents/DTU_resource/Thesis/Incremental-learning-EL/src/data-withtemp.csv'
categoryFeatures=['dayOfYear','hourOfDay', 'dayOfWeek', 'holiday', 'weekend']

In [60]:
df = pd.read_csv(dataPath)
df = df.sort_values(by=['datetime'])
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['temp']

In [61]:
scaler = pre.StandardScaler() 
scaler.fit(df[df['datetime']< '2020-01-01']['meter'].values.reshape(-1, 1))

meter = scaler.transform(df['meter'].values.reshape(-1, 1))
df['meter'] = list(meter.ravel())

df = shiftData(12, 24, 'meter')
df = df.reset_index()
del df['index']
df.head()

,datetime,meter,hourOfDay,dayOfWeek,dayOfYear,holiday,prev_meter12,prev_meter13,prev_meter14,prev_meter15,prev_meter16,prev_meter17,prev_meter18,prev_meter19,prev_meter20,prev_meter21,prev_meter22,prev_meter23,prev_meter24
0,2019-01-02 00:00:00,-0.778512,0,2,2,0,0.888003,0.773931,0.338515,0.240032,-0.039775,-0.445899,-0.785941,-0.952268,-1.059915,-0.808519,-0.764492,-0.352465,-0.122258
1,2019-01-02 01:00:00,-1.018446,1,2,2,0,0.811891,0.888003,0.773931,0.338515,0.240032,-0.039775,-0.445899,-0.785941,-0.952268,-1.059915,-0.808519,-0.764492,-0.352465
2,2019-01-02 02:00:00,-1.073687,2,2,2,0,0.617414,0.811891,0.888003,0.773931,0.338515,0.240032,-0.039775,-0.445899,-0.785941,-0.952268,-1.059915,-0.808519,-0.764492
3,2019-01-02 03:00:00,-1.175142,3,2,2,0,0.850153,0.617414,0.811891,0.888003,0.773931,0.338515,0.240032,-0.039775,-0.445899,-0.785941,-0.952268,-1.059915,-0.808519
4,2019-01-02 04:00:00,-0.974736,4,2,2,0,1.349519,0.850153,0.617414,0.811891,0.888003,0.773931,0.338515,0.240032,-0.039775,-0.445899,-0.785941,-0.952268,-1.059915


In [73]:
#split train test
features = [x for x in data_test.columns if x not in ['meter','datetime']]

data_train = df[df['datetime']< '2020-01-01']
data_test = df[df['datetime'] >= '2020-01-01']

X_train = data_train[features].to_numpy()
y_train = data_train['meter'].to_numpy()

X_test = data_test[features].to_numpy()
y_test = data_test['meter'].to_numpy()


17400
8712


(8712,)

In [74]:
svr = SVR(kernel='linear', C=10, gamma='auto')
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.8498857189390776

# LSTM

In [ ]:
class OLSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(OLSTM, self).__init__() 
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # print(x.size())
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out



model = model
lr = learningRate
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = Adam(model.parameters(), lr=lr)

for e in epoch_num:
    outputs = self.model(X)
    self.optimizer.zero_grad()

    loss = self.criterion(outputs, y)
    loss.backward()
    self.optimizer.step()


In [ ]:

    def _initDeepData(self):
        df = self._readData(self.dataPath)
        df = df.sort_values(by=['datetime'])
        trainSet = df['meter'].values.reshape(-1, 1)
        trainData = self.deepScaler.fit_transform(trainSet)
        x, y = self._slidingWindows(trainData)
        times = df[self.seqLength+self.lagLength:]['datetime']
        # times = times
        return x, y, times

    
    def _slidingWindows(self, data):
        x = []
        y = []
        for i in range(len(data)-self.seqLength-self.lagLength):
            _x = data[i:(i+self.seqLength)]
            _y = data[i+self.seqLength+self.lagLength]
            x.append(_x)
            y.append(_y)
        return np.array(x),np.array(y)


    def _initData(self):
        df = self._readData(self.dataPath)
        df = df.sort_values(by=['datetime'])
        df = self._shiftData(df=df)
        df = df.reset_index()
        features = [f for f in list(df.columns) if f not in self.removeFeatures]
        df = df[features]   
        return self._splitData(df)


    def _shiftData(self, df):
        for i in range(self.shiftRange[0], self.shiftRange[1] + 1):
            for f in self.shiftFeatures:
                df['prev_' + f + str(i)] = df[f].shift(periods=i)
        return df.dropna().copy()


    def _splitData(self, df):    
        scaleFeatures = [f for f in df.columns if f not in self.categoryFeatures and f not in ['datetime']]
        nonscaleFeatures = [f for f in df.columns if f in self.categoryFeatures]
        return df[scaleFeatures].copy(), df[nonscaleFeatures].copy(), df['meter'].copy(), df['datetime'].copy()


    def _readData(self, dataPath):
        try:
            df = pd.read_csv(dataPath)
        except Exception as e:
            logging.error(e)
            raise e
        return df.dropna()
